# CellOracle-based perturbation prediction

Notebook for predicts TF perturbation effects with CellOracle.

## Library imports

In [1]:
import numpy as np
import pandas as pd

import celloracle as co
import scanpy as sc
import scvelo as scv
from celloracle.applications import Gradient_calculator, Oracle_development_module
from scvelo import logging as logg

from rgv_tools import DATA_DIR
from rgv_tools.perturbation import split_elements,combine_elements,TFScanning_perturbation_co,Multiple_TFScanning_perturbation_co
from rgv_tools.perturbation import get_list_name

## Constants

In [2]:
DATASET = "zebrafish"

In [3]:
SAVE_DATA = False
if SAVE_DATA:
    (DATA_DIR / DATASET / "results").mkdir(parents=True, exist_ok=True)

In [4]:
TERMINAL_STATES = [
    "mNC_head_mesenchymal",
    "mNC_arch2",
    "mNC_hox34",
    "Pigment",
]

In [5]:
single_ko = ["rarga","rxraa","nr2f5","fli1a","tfec","elk3","mitfa","ets1","nr2f2","elf1","ebf3a"]
multiple_ko = ["fli1a_elk3","tfec_mitfa_bhlhe40","mitfa_tfec","mitfa_tfec_tfeb"]

## Data loading

In [13]:
adata = sc.read_h5ad(DATA_DIR / DATASET / "raw" / "adata_zebrafish_preprocessed.h5ad")

In [14]:
df = pd.read_csv(DATA_DIR / DATASET / "raw" / "eRegulon_metadata_all.csv", index_col=0)

## Data processing

In [15]:
df.loc[:, "Target"] = [f"{a}*{b}" for a, b in zip(df.loc[:, "Region"].tolist(), df.loc[:, "Gene"].tolist())]
df = df.loc[:, ["TF", "Gene", "Target"]]

regulators = df["TF"].unique()
targets = df["Target"].unique()

# Create an empty binary matrix
binary_matrix = pd.DataFrame(0, columns=regulators, index=targets)

# Fill the binary matrix based on the relationships in the CSV file
for _, row in df.iterrows():
    binary_matrix.at[row["Target"], row["TF"]] = 1

original_list = binary_matrix.index.tolist()
peak = [item.split("*")[0] for item in original_list]
target = [item.split("*")[1] for item in original_list]

binary_matrix.loc[:, "peak_id"] = peak
binary_matrix.loc[:, "gene_short_name"] = target

binary_matrix = binary_matrix[
    ["peak_id", "gene_short_name"] + [col for col in binary_matrix if col not in ["peak_id", "gene_short_name"]]
]
binary_matrix = binary_matrix.reset_index(drop=True)

In [16]:
scv.pp.moments(adata, n_pcs=50, n_neighbors=30)

computing neighbors
    finished (0:00:06) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:00) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)


## CellOracle pipeline

In [17]:
adata.X = adata.layers["matrix"].copy()
oracle = co.Oracle()
oracle.import_anndata_as_raw_count(adata=adata, cluster_column_name="cell_type", embedding_name="X_umap")

oracle.import_TF_data(TF_info_matrix=binary_matrix)

8012 genes were found in the adata. Note that Celloracle is intended to use around 1000-3000 genes, so the behavior with this number of genes may differ from what is expected.


In [18]:
oracle.perform_PCA()

In [19]:
n_comps = np.where(np.diff(np.diff(np.cumsum(oracle.pca.explained_variance_ratio_)) > 0.002))[0][0]

In [20]:
n_comps = min(n_comps, 50)

In [21]:
n_cell = oracle.adata.shape[0]
print(f"cell number is :{n_cell}")

k = int(0.025 * n_cell)
print(f"Auto-selected k is :{k}")

cell number is :697
Auto-selected k is :17


In [22]:
oracle.knn_imputation(n_pca_dims=n_comps, k=k, balanced=True, b_sight=k * 8, b_maxl=k * 4, n_jobs=4)

In [23]:
links = oracle.get_links(cluster_name_for_GRN_unit="cell_type", alpha=10, verbose_level=10)

  0%|          | 0/13 [00:00<?, ?it/s]

Inferring GRN for NC_trunk...


  0%|          | 0/1925 [00:00<?, ?it/s]

Inferring GRN for NPB_hox3...


  0%|          | 0/1925 [00:00<?, ?it/s]

...

  0%|          | 0/1925 [00:00<?, ?it/s]

Inferring GRN for mNC_nohox...


  0%|          | 0/1925 [00:00<?, ?it/s]

Inferring GRN for mNC_vagal...


  0%|          | 0/1925 [00:00<?, ?it/s]

In [24]:
links.filter_links(p=0.001, weight="coef_abs", threshold_number=2000)
links.get_network_score()

In [25]:
links.filter_links()
oracle.get_cluster_specific_TFdict_from_Links(links_object=links)
oracle.fit_GRN_for_simulation(alpha=10, use_cluster_specific_TFdict=True)

  0%|          | 0/13 [00:00<?, ?it/s]

### Reference vector field definition

In [26]:
## calculate pseudotime
scv.tl.recover_dynamics(adata, var_names=adata.var_names, n_jobs=4)
scv.tl.velocity(adata, mode="dynamical")
scv.tl.latent_time(adata, min_likelihood=None)

recovering dynamics (using 4/128 cores)


  0%|          | 0/8012 [00:00<?, ?gene/s]

    finished (0:05:21) --> added 
    'fit_pars', fitted parameters for splicing dynamics (adata.var)
computing velocities
    finished (0:00:02) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)
computing velocity graph (using 1/128 cores)


  0%|          | 0/697 [00:00<?, ?cells/s]

    finished (0:00:00) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)
computing terminal states
    identified 0 region of root cells and 1 region of end points .
    finished (0:00:00) --> added
    'root_cells', root cells of Markov diffusion process (adata.obs)
    'end_points', end points of Markov diffusion process (adata.obs)
computing latent time using root_cells as prior
    finished (0:00:00) --> added 
    'latent_time', shared time (adata.obs)


In [27]:
## use the velocity latent time inferred by scVelo to create gradient field
n_grid = 40
min_mass = 1.5
oracle.adata.obs["Pseudotime"] = adata.obs["latent_time"].copy()
gradient = Gradient_calculator(oracle_object=oracle, pseudotime_key="Pseudotime")
gradient.calculate_p_mass(smooth=0.8, n_grid=n_grid, n_neighbors=30)
gradient.calculate_mass_filter(min_mass=min_mass, plot=True)
gradient.transfer_data_into_grid(args={"method": "polynomial", "n_poly": 3}, plot=True)
gradient.calculate_gradient()

In [28]:
scale_dev = 40
gradient.visualize_results(scale=scale_dev, s=5)

In [29]:
cell_idx_pigment = np.where(oracle.adata.obs["cell_type"].isin(["Pigment"]))[0]

cell_idx_hox34 = np.where(oracle.adata.obs["cell_type"].isin(["mNC_hox34"]))[0]

cell_idx_arch2 = np.where(oracle.adata.obs["cell_type"].isin(["mNC_arch2"]))[0]

cell_idx_mesenchymal = np.where(oracle.adata.obs["cell_type"].isin(["mNC_head_mesenchymal"]))[0]

In [30]:
index_dictionary = {
    "Pigment": cell_idx_pigment,
    "mNC_hox34": cell_idx_hox34,
    "mNC_arch2": cell_idx_arch2,
    "mNC_head_mesenchymal": cell_idx_mesenchymal,
}

## Perturbation prediction

### single knock-out

In [31]:
n_neighbors = 30

In [32]:
single_ko= set(single_ko).intersection(adata.var_names)
single_ko = list(single_ko)

In [33]:
## celloracle perturbation
d = TFScanning_perturbation_co(adata, 8, "cell_type", TERMINAL_STATES, single_ko,oracle,gradient,index_dictionary,n_neighbors)

2025-08-29 20:21:01,813 - INFO - Done tfec
2025-08-29 20:21:23,457 - INFO - Done ets1
2025-08-29 20:21:47,008 - INFO - Done nr2f5
2025-08-29 20:22:08,972 - INFO - Done elf1
2025-08-29 20:22:31,299 - INFO - Done ebf3a
2025-08-29 20:22:52,980 - INFO - Done elk3
2025-08-29 20:23:14,410 - INFO - Done nr2f2
2025-08-29 20:23:35,687 - INFO - Done rxraa
2025-08-29 20:23:57,520 - INFO - Done fli1a
2025-08-29 20:24:19,095 - INFO - Done rarga
2025-08-29 20:24:41,067 - INFO - Done mitfa


In [34]:
coef = pd.DataFrame(np.array(d["coefficient"]))
coef.index = d["TF"]
coef.columns = get_list_name(d["coefficient"][0])

### multiple knock-out

In [35]:
multiple_ko_list = split_elements(multiple_ko)

In [36]:
d = Multiple_TFScanning_perturbation_co(adata, 8, "cell_type", TERMINAL_STATES, multiple_ko_list,oracle,gradient,index_dictionary,n_neighbors)

2025-08-29 20:25:03,273 - INFO - Done fli1a_elk3
2025-08-29 20:25:27,053 - INFO - Done tfec_mitfa_bhlhe40
2025-08-29 20:25:48,972 - INFO - Done mitfa_tfec
2025-08-29 20:26:11,302 - INFO - Done mitfa_tfec_tfeb


In [37]:
coef_multiple = pd.DataFrame(np.array(d["coefficient"]))
coef_multiple.index = d["TF"]
coef_multiple.columns = get_list_name(d["coefficient"][0])

## Save dataset

In [41]:
if SAVE_DATA:
    coef.to_csv(DATA_DIR / DATASET / "results" / "celloracle_perturb_single.csv")
    coef_multiple.to_csv(DATA_DIR / DATASET / "results" / "celloracle_perturb_multiple.csv")